# Práctica 1. Diferenciación de entidades.
## Autores: 
     Raúl Sánchez Martín
     Ignacio Arias Barra

In [3]:
import re, pprint, os, numpy
# -*- coding: utf-8 -*-
import nltk
import string
path_to_append = '/media/nacho/f8371289-0f00-4406-89db-d575f3cdb35e/Master/Trimestre 2/RIM/nltk_data'
nltk.data.path.append(path_to_append)
from sklearn.metrics.cluster import *
from sklearn.cluster import AgglomerativeClustering
from nltk.cluster import GAAClusterer
from sklearn.metrics.cluster import adjusted_rand_score


## Funciones

### Código origen
Lectura de fichero:

In [4]:
def read_file(file):
    myfile = open(file,"r")
    data = ""
    lines = myfile.readlines()
    for line in lines:
        data = data + line
    myfile.close
    return data


Clustering:

In [5]:
def cluster_texts(texts, clustersNumber, distance):
    # Convierte textos en una coleccion
    # Load the list of texts into a TextCollection object.
    collection = nltk.TextCollection(texts)    
#     print("Created a collection of", len(collection), "terms.")
    
    # Get a list of unique terms
    unique_terms = list(set(collection))
#     print("Unique terms found: ", len(unique_terms))

    ### And here we actually call the function and create our array of vectors.
    # TF mide la frecuencia en los textos.
    # Mira de los terminos unicos, cuantas veces aparece en el documento. No mira cuantas veces aparece en la coleccion
    # Hay otras medidas, como TF-IDF que son mas precisas porque tambien miran cuantas veces aparece en la coleccion
    vectors = [numpy.array(TF(f,unique_terms, collection)) for f in texts]
    # print("Vectors created.")
    # print(vectors)

    # initialize the clusterer
    clusterer = GAAClusterer(clustersNumber)
    clusters = clusterer.cluster(vectors, True)
    # Estas lineas siguientes comentadas es lo mismo pero con otra libreria, la llamada scikit-learn
    # clusterer = AgglomerativeClustering(n_clusters=clustersNumber,
    #                                  linkage="average", affinity=distanceFunction)
    #clusters = clusterer.fit_predict(vectors)

    return clusters

Frecuencia de términos únicos en un documento

In [6]:
# Function to create a TF vector for one document. For each of
# our unique words, we have a feature which is the tf for that word
# in the current document
def TF(document, unique_terms, collection):
    word_tf = []
    for word in unique_terms:
        word_tf.append(collection.tf(word, document))
    return word_tf

### Nuevo código

*EVALUACIÓN DIFERENTES TAMAÑOS DE CLUSTER*

In [7]:
def eval_test(top_test, priori_groups, reference):
    score = 0
    curr_score = -9999999
    curr_priori_groups = 1
    for n in range(top_test):
        test = cluster_texts(texts,priori_groups,distanceFunction)
        # print("test: ", test)
        # Evaluation
        score = adjusted_rand_score(reference, test)

        print("group: ", priori_groups)
        print("rand_score: ", score)

        if score >= curr_score:
            curr_score = score
            curr_priori_groups = priori_groups

        priori_groups+=1

    # Gold Standard
    # el documento que meto el primero sera igual que los que esten en todos los ceros
    print("\nBest agroupation: ", curr_priori_groups)
    print("Best score: ", curr_score)
    print("reference: ", reference)
    return curr_priori_groups

*USO DE ENTIDADES*

*ELIMINAR PALABRAS VACÍAS*

In [8]:
from nltk.corpus import stopwords


# # Abrimos el archivo.
# with open('./Thomas_Baker/005.txt', 'r', encoding="utf8") as f:
# # with open('./Thomas_Baker/036.txt', 'r', encoding="utf8") as f:
#     text = f.read()
# f.close()


def delete_stopwords(text,lan='en'):

    if lan == 'en':        
        # Obtenemos las sentencias del texto.
        sentences = nltk.sent_tokenize(text)
        # Obtenemos las "stopwords" del inglés.
        stop = set(stopwords.words('english'))

    elif lan == 'es':        
        # Elegimos el tokenizador para el español de la NLTK.
        spanish_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')
        # Obtenemos las sentencias del texto.
        sentences = spanish_tokenizer.tokenize(text)        

        # Obtenemos las "stopwords" del español.
        stop = set(stopwords.words('spanish'))
      
    print("Frases originales: ")
    print(sentences)
    
    stopw = []
    for i in stop:
        stopw.append(i)
    print("Las stop words en lan = " + lan)
    print(stopw)
    print()

    # Eliminamos las palabras que coinciden con alguna "stopword".
    print("Frases procesadas sin stopwords: ")
    for sentence in sentences:
        non_stop_sentence = ""
        for word in sentence.lower().split():
            if word not in stop:
                non_stop_sentence = non_stop_sentence + word + " "
        # Imprimimos las sentencias procesadas.
        print(non_stop_sentence)
    return non_stop_sentence

# espanol = delete_stopwords(text,lan='es')
# ingles = delete_stopwords(text)

*ELIMINAR EL NOMBRE DE LA PERSONA EN LOS TEXTOS*

*REPRESENTACIÓN DE NGRAMAS A NIVEL DE PALABRAS*

(trigramas es poner casa-perro-coche en las ocurrencias por texto),
 se puede hacer los ngramas con nltk y scikit-learn

*STEAMMING*

In [9]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

# # Abrimos el archivo.
# with open('./Thomas_Baker/005.txt', 'r', encoding="utf8") as f:
# with open('./Thomas_Baker/036.txt', 'r', encoding="utf8") as f:
#     text = f.read()
# f.close()

def stemming(text, lan='en'):
    stemmeds = []
    if lan == 'en':
        # Steamer ingles
        stemmer = PorterStemmer()        
    elif lan == 'es':
        # Stemer espanol        
        stemmer = SnowballStemmer("spanish")
    
    # Obtenemos los tokens del texto.
    tokens = nltk.word_tokenize(text)
        
    # Para cada token del texto obtenemos su raíz.
    for token in tokens:
        stemmed = stemmer.stem(token)
        stemmeds.append(stemmed)
    # Escribimos el resultado para compararlo con las palabras originales.
    print("Palabras originales: ")
    print(tokens)
    print("Palabras procesadas con Snowball: ")
    print(stemmeds)
    
# stemming(text)

*EXPLORAR CONTENIDO DE LAS PÁGINAS WEB*

## Inicio programa

In [10]:
# Folder with all texts
folder = "Thomas_Baker"
# Empty list to hold text documents.
texts = []

listing = os.listdir(folder)
for file in listing:
    if file.endswith(".txt"):
        url = folder+"/"+file
        f = open(url,encoding="latin-1");
        raw = f.read()
        f.close()
        tokens = nltk.word_tokenize(raw)
        text = nltk.Text(tokens)
        texts.append(text)

print("Prepared ", len(texts), " documents...")
print("They can be accessed using texts[0] - texts[" + str(len(texts)-1) + "]")

# Similarity distance
distanceFunction ="cosine"
# distanceFunction = "euclidean"

# iterations for the evaluation of clusters
top_test = 10
priori_groups = 1

# Final solution for comparing in tests
reference = [0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 2, 0, 3, 3, 0, 1, 2, 0, 1]
groups = eval_test(top_test, priori_groups, reference)

Prepared  19  documents...
They can be accessed using texts[0] - texts[18]
group:  1
rand_score:  0.0
group:  2
rand_score:  -0.106489184692
group:  3
rand_score:  -0.0900678593461
group:  4
rand_score:  -0.100195185426
group:  5
rand_score:  -0.0928104575163
group:  6
rand_score:  0.010719754977
group:  7
rand_score:  0.0477326968974
group:  8
rand_score:  -0.0145631067961
group:  9
rand_score:  -0.0164609053498
group:  10
rand_score:  -0.059950041632

Best agroupation:  7
Best score:  0.0477326968974
reference:  [0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 2, 0, 3, 3, 0, 1, 2, 0, 1]


In [11]:
# En la memoria se ha de justificar cada cambio propuesto.

# Se ha de comentar los resultados obtenidos.

# Siempre buscando una mejora en la clusterización